<a href="https://colab.research.google.com/github/shreyanknp/DA1/blob/main/ML_project_novelty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Load the dataset (replace with your dataset path)
path='/content/drive/MyDrive/Datasets/pwr_raw.csv'
data = pd.read_csv(path, sep=r'\s+', header=None)

# Create new column names
column_names = [f'rod_{i}' for i in range(1, 40)] + ['k_inf', 'pppf']

print(data.head())

# Assign the new column names to the DataFrame
data.columns = column_names

# Check the updated column names
print(data.columns)

print(data.head())


        0         1         2         3         4         5         6   \
0  1.32630  1.862086  3.562615  1.112359  4.208065  1.412679  2.271031   
1  1.34149  1.786558  2.354766  1.047929  2.268974  4.637777  3.793194   
2  1.33034  1.826187  3.037997  4.719517  1.502280  3.644736  3.575727   
3  1.35653  1.739225  4.412804  2.220680  3.132738  1.327124  4.759890   
4  1.30190  1.955829  2.790669  0.798177  1.231750  1.977844  2.393930   

         7         8         9   ...        31        32        33        34  \
0  3.950614  3.627032  4.600332  ...  3.965987  4.030999  0.826347  2.088521   
1  2.859702  4.361058  3.592485  ...  2.228000  4.837370  2.846538  3.413201   
2  1.337686  2.380347  4.503771  ...  2.615278  3.757999  3.161658  4.996056   
3  2.882082  3.663430  4.531524  ...  3.899026  1.018926  4.379308  3.280298   
4  4.738126  1.364743  3.391665  ...  3.641512  1.564076  2.178406  2.754888   

         35        36        37        38        39        40  
0  4.08855

In [ ]:
import numpy as np
import pandas as pd
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load the dataset (replace with your dataset path)
#data = pd.read_csv('path_to_your_dataset.csv')

# Separate features and target variables
X = data.drop(columns=['k_inf', 'pppf'])  # Fuel rod enrichment values
y = data[['k_inf', 'pppf']]  # Target variables

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model for optimization (Random Forest Regressor)
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Define the search space for Bayesian optimization
param_space = {
    'max_depth': Integer(1, 20),  # max_depth must be an integer between 1 and 20
    'min_samples_split': Integer(2, 10),  # min_samples_split as an integer between 2 and 10
    'min_samples_leaf': Real(0.01, 0.1, 'uniform'),  # min_samples_leaf as a float between 0.01 and 0.1
}

# Initialize Bayesian Optimization with the corrected param_space
opt = BayesSearchCV(model, param_space, n_iter=50, cv=3, random_state=42)

# Fit the model to the training data using Bayesian Optimization
opt.fit(X_train, y_train)

# Best parameters found through Bayesian Optimization
print("Best parameters:", opt.best_params_)




Best parameters: OrderedDict([('max_depth', 20), ('min_samples_leaf', 0.01), ('min_samples_split', 2)])
